In [ ]:
from getpass import getpass
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import db_builder.db_handler as dbh
import logging
import seaborn as sns 

logging.getLogger('sqlalchemy').setLevel(logging.WARNING)
db_params = {
    'user': 'postgres',
    'password': getpass('Please enter DB pw'),  # enter your DB password
    'host': 'localhost',  # 'localhost' or IP address
    'port': '5432',  # '5432'
    'database': 'ttdatabase',  #tensionTerminator
}
toolcheck = dbh.DB_Conn(db_params)
toolcheck.connect()
engine = toolcheck.get_engine()

In [ ]:
def table_to_df(tabel_name: str):
    return pd.read_sql_table(tabel_name, con=engine)

In [ ]:
def get_tools(id):
    tools_df = table_to_df('bodyside')
    tool_loc = tools_df.loc[tools_df['id'] == id].values
    return f"{tool_loc[0][1]}"

In [ ]:
bodyside_loop_association_df = table_to_df('bodyside_loop_association').dropna()
bodyside_df = table_to_df('bodyside').dropna()

In [ ]:
bodyside_df

In [ ]:
count = bodyside_df.groupby(['side', 'time_in_use']).size().reset_index(name='Count')
count

In [ ]:
bodyside_df['time_in_use'] = pd.to_datetime(bodyside_df['time_in_use'], format='%H:%M:%S')
bodyside_df['time_in_use_seconds'] = bodyside_df['time_in_use'].dt.hour * 3600 + bodyside_df['time_in_use'].dt.minute * 60 + bodyside_df['time_in_use'].dt.second

bodyside_df

In [ ]:
time_sum_per_bodyside = bodyside_df.groupby('side')['time_in_use_seconds'].sum().reset_index()
time_sum_per_bodyside

In [ ]:
colors = sns.color_palette('husl', n_colors=len(time_sum_per_bodyside))

plt.figure(figsize=(12, 10))

plt.bar(time_sum_per_bodyside['side'], time_sum_per_bodyside['time_in_use_seconds']/60, color=colors)

plt.xlabel('Tools')
plt.ylabel('Minuten')
plt.title(f"Count of each Tool, summarized time: {time_sum_per_bodyside['time_in_use_seconds'].sum()/60} min")

plt.show()